# データ分析サンプル

このNotebookはテーブルデータの基本的な分析手法を示します。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Matplotlib設定
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. サンプルデータの作成

分析用のサンプルデータを作成します。

In [ ]:
# シード値を設定して再現可能にする
np.random.seed(42)

# サンプルデータの作成
n_samples = 1000

# 顧客データのシミュレーション
data = {
    'customer_id': range(1, n_samples + 1),
    'age': np.random.normal(35, 12, n_samples).astype(int),
    'income': np.random.exponential(50000, n_samples),
    'spending_score': np.random.normal(50, 25, n_samples),
    'purchase_frequency': np.random.poisson(3, n_samples),
    'category': np.random.choice(['A', 'B', 'C', 'D'], n_samples),
    'satisfaction': np.random.uniform(1, 5, n_samples)
}

# データフレームの作成
df = pd.DataFrame(data)

# 年齢の制限
df['age'] = np.clip(df['age'], 18, 80)

# スペンディングスコアの制限
df['spending_score'] = np.clip(df['spending_score'], 1, 100)

# 収入とスペンディングスコアの相関を追加
df['spending_score'] += df['income'] / 2000
df['spending_score'] = np.clip(df['spending_score'], 1, 100)

print(f"データセットの形状: {df.shape}")
df.head()

## 2. 基本的な統計情報の確認

In [ ]:
# 基本統計情報
print("=== 基本統計情報 ===")
print(df.describe())

print("\n=== データ型情報 ===")
print(df.info())

print("\n=== 欠損値の確認 ===")
print(df.isnull().sum())

## 3. データ可視化

In [ ]:
# 数値データの分布を確認
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('数値変数の分布', fontsize=16)

numerical_cols = ['age', 'income', 'spending_score', 'purchase_frequency', 'satisfaction']

for i, col in enumerate(numerical_cols):
    row = i // 3
    col_idx = i % 3
    
    axes[row, col_idx].hist(df[col], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[row, col_idx].set_title(f'{col}の分布')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('頻度')

# 空のサブプロットを削除
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

In [ ]:
# カテゴリ変数の可視化
plt.figure(figsize=(8, 6))
df['category'].value_counts().plot(kind='bar', color='lightcoral')
plt.title('カテゴリ別顧客数')
plt.xlabel('カテゴリ')
plt.ylabel('顧客数')
plt.xticks(rotation=0)
plt.show()

## 4. 相関分析

In [ ]:
# 相関行列の計算と可視化
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('変数間の相関行列')
plt.tight_layout()
plt.show()

print("強い相関関係:")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            print(f"{correlation_matrix.columns[i]} と {correlation_matrix.columns[j]}: {corr_val:.3f}")

## 5. 散布図による関係性の可視化

In [ ]:
# 収入とスペンディングスコアの関係
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['income'], df['spending_score'], c=df['age'], 
                     cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='年齢')
plt.xlabel('収入')
plt.ylabel('スペンディングスコア')
plt.title('収入とスペンディングスコアの関係（年齢別）')
plt.show()

## 6. カテゴリ別分析

In [ ]:
# カテゴリ別のボックスプロット
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('カテゴリ別データ分布', fontsize=16)

variables = ['income', 'spending_score', 'age', 'satisfaction']

for i, var in enumerate(variables):
    row = i // 2
    col = i % 2
    
    sns.boxplot(data=df, x='category', y=var, ax=axes[row, col])
    axes[row, col].set_title(f'カテゴリ別{var}分布')

plt.tight_layout()
plt.show()

## 7. クラスター分析

In [ ]:
# クラスター分析用のデータ準備
features_for_clustering = ['income', 'spending_score', 'age']
X = df[features_for_clustering]

# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-meansクラスタリング
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# クラスター結果をデータフレームに追加
df['cluster'] = clusters

print("クラスター別顧客数:")
print(df['cluster'].value_counts().sort_index())

In [ ]:
# クラスター結果の可視化
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['income'], df['spending_score'], c=df['cluster'], 
                     cmap='Set1', alpha=0.7)
plt.colorbar(scatter, label='クラスター')
plt.xlabel('収入')
plt.ylabel('スペンディングスコア')
plt.title('K-meansクラスタリング結果')

# クラスター中心点をプロット
centers = scaler.inverse_transform(kmeans.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x', s=200, linewidths=3)

plt.show()

## 8. 回帰分析

In [ ]:
# 線形回帰：収入からスペンディングスコアを予測
X = df[['income', 'age']].values
y = df['spending_score'].values

# 訓練・テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 線形回帰モデルの学習
model = LinearRegression()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデル評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"平均二乗誤差 (MSE): {mse:.2f}")
print(f"決定係数 (R²): {r2:.3f}")
print(f"\n回帰係数:")
print(f"収入: {model.coef_[0]:.6f}")
print(f"年齢: {model.coef_[1]:.6f}")
print(f"切片: {model.intercept_:.2f}")

In [ ]:
# 予測結果の可視化
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('実際のスペンディングスコア')
plt.ylabel('予測されたスペンディングスコア')
plt.title(f'線形回帰の予測結果 (R² = {r2:.3f})')
plt.show()

## 9. インタラクティブな可視化（Plotly）

In [ ]:
# Plotlyを使用したインタラクティブな散布図
fig = px.scatter(df, x='income', y='spending_score', 
                 color='cluster', size='age',
                 hover_data=['purchase_frequency', 'satisfaction'],
                 title='顧客データのインタラクティブ散布図',
                 labels={'income': '収入', 'spending_score': 'スペンディングスコア'})

fig.show()

## 10. まとめ

このNotebookでは以下の分析を実行しました：

1. **基本統計情報の確認**: データの概要と分布を把握
2. **データ可視化**: ヒストグラム、ボックスプロットによる分布の確認
3. **相関分析**: 変数間の関係性を数値とヒートマップで確認
4. **カテゴリ別分析**: グループごとの特徴を比較
5. **クラスター分析**: K-meansによる顧客セグメンテーション
6. **回帰分析**: 線形回帰による予測モデル構築
7. **インタラクティブ可視化**: Plotlyによる動的なグラフ作成

### 次のステップ
- 実際のデータを読み込んでこの分析手法を適用
- より高度な機械学習手法の試行
- 時系列データがある場合の時系列分析
- A/Bテストや統計的検定の実装